In [10]:
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
import time
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from scipy import sparse, integrate
import concurrent.futures
import scipy.sparse.linalg as sla

sigma_0 = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])

tau_0 = sigma_0
tau_x = sigma_x
tau_y = sigma_y
tau_z = sigma_z

s_0 = sigma_0
s_x = sigma_x
s_y = sigma_y
s_z = sigma_z


Lx = 11
Ly = 11

def make_lead():
    
    lat = kwant.lattice.cubic(1,norbs=2)
    
    sym_lead = kwant.TranslationalSymmetry((1, 0, 0))
    lead = kwant.Builder(sym_lead)
    
    def lead_shape(pos):
        x,y,z = pos
        return -Lx<= x <=Lx and -Ly<= y <= Ly and 0<= z <= 1
        
    
    def onsite (site,t_dx,t_dy,mu_d_down,mu_d_up,delta_dx_down,delta_dx_up,delta_dy_down,delta_dy_up,phi):
        x,y,z = site.pos
        if z == 0:
            return (2*t_dx+2*t_dy-mu_d_down)*tau_z
        else:
            return (2*t_dx+2*t_dy-mu_d_up)*tau_z
                    
    
    def hopping_x(site1,site2,t_dx,delta_dx_down,delta_dx_up,phi):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        if z1==0 and z2==0:
            return  -t_dx*tau_z-(delta_dx_down*cos(phi*pi)*tau_x+delta_dx_down*sin(phi*pi)*tau_y)
        else:
            return -t_dx*tau_z-delta_dx_up*tau_x
        
    
    def hopping_y(site1,site2,t_dy,delta_dy_down,delta_dy_up,phi):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        if z1==0 and z2==0:
            return  -t_dy*tau_z+(delta_dy_down*cos(phi*pi)*tau_x+delta_dy_down*sin(phi*pi)*tau_y)
        else:
            return -t_dy*tau_z+delta_dy_up*tau_x
        
        
    def hopping_z(site1,site2,t_couple):
        return t_couple *tau_z


    lead[lat.shape(lead_shape, (0,0,0))] = onsite
    lead[kwant.builder.HoppingKind((1, 0, 0), lat, lat)] = hopping_x
    lead[kwant.builder.HoppingKind((0, 1, 0), lat, lat)] = hopping_y
    lead[kwant.builder.HoppingKind((0, 0, 1), lat, lat)] = hopping_z
    
    return lead


t_dx = 1
t_dy = 1

mu_d_down = 2.5
mu_d_up = 2.5


p_down = 1
delta_dx_down = 0.2
delta_dy_down = (1-2*p_down)*0.2

p_up = 0
delta_dx_up = 0.2 
delta_dy_up = (1-2*p_up)*0.2

t_couple = 0.4


def plot_band(phi):
    
    
    lead = make_lead().finalized()
    parameters = dict(t_dx=t_dx,t_dy=t_dy,mu_d_up=mu_d_up,mu_d_down=mu_d_down,delta_dx_down=delta_dx_down,delta_dx_up=delta_dx_up,\
                      delta_dy_down=delta_dy_down,delta_dy_up=delta_dy_up,phi=phi,t_couple=t_couple) 
    
    bands = kwant.physics.Bands(lead,params = parameters)
    momenta = np.linspace(-np.pi, np.pi, 1001)
    energies = [np.sort(bands(k)) for k in momenta]
    
    pyplot.figure()
    pyplot.plot(momenta/np.pi, energies,'black')
    pyplot.xlabel(r'$k_{x}/\pi$')
    pyplot.ylabel(r'Energies')
    pyplot.show()


def main():
    start = time.time() 

    plot_band(0.)
    
    
    end = time.time()     
    print('Running time: %s seconds'%(end-start))
# Call the main function if the script gets executed (as opposed to imported).
# See <http://docs.python.org/library/__main__.html>.
if __name__ == '__main__':
    main()
    


Using matplotlib backend: Qt5Agg
Running time: 1.0776786804199219 seconds
